In [ ]:
# Instalar ultralytics si no está
!pip install ultralytics -q

In [ ]:
# Verificar GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Preparar Dataset

Después de capturar imágenes con `capture_dataset.py`, sube las imágenes a Roboflow:

1. Ve a [roboflow.com](https://roboflow.com) y crea cuenta gratis
2. Crea proyecto: **Object Detection** → **Oriented Bounding Boxes (OBB)**
3. Sube las imágenes de `dataset/images/`
4. Etiqueta cada figurita:
   - Dibuja un rectángulo rotado alrededor de la figurita
   - El lado más largo del rectángulo debe apuntar hacia donde mira
   - Clase: `miniature`
5. Genera dataset → Exportar → YOLO OBB → Descargar código

In [ ]:
# Configurar rutas del dataset
# Modifica esta ruta al descargar tu dataset de Roboflow
DATASET_PATH = "../dataset"  # Carpeta con data.yaml

import os
from pathlib import Path

dataset_yaml = Path(DATASET_PATH) / "data.yaml"
if dataset_yaml.exists():
    print(f"✅ Dataset encontrado: {dataset_yaml}")
    with open(dataset_yaml) as f:
        print(f.read())
else:
    print(f"❌ No encontrado: {dataset_yaml}")
    print("Descarga tu dataset de Roboflow y extráelo en 'dataset/'")

## 2. Entrenar Modelo

In [ ]:
from ultralytics import YOLO

# Cargar modelo base nano-obb (el más ligero con rotación)
model = YOLO('yolov8n-obb.pt')

# Entrenar
results = model.train(
    data=str(dataset_yaml),
    epochs=100,           # 100 epochs suelen ser suficientes
    imgsz=640,            # Tamaño de imagen
    batch=16,             # Ajustar según VRAM (16 para 8GB)
    device=0,             # GPU 0
    patience=20,          # Early stopping
    save=True,
    project='runs/train',
    name='miniatures_obb',
    
    # Augmentaciones para mejorar generalización
    degrees=180,          # Rotación completa (figuritas pueden mirar a cualquier lado)
    flipud=0.5,           # Flip vertical
    fliplr=0.5,           # Flip horizontal
    mosaic=0.5,           # Mosaic augmentation
    scale=0.3,            # Variación de escala
)

In [ ]:
# Ver métricas de entrenamiento
from IPython.display import Image, display

# Mostrar curvas de entrenamiento
results_dir = Path('runs/train/miniatures_obb')
if (results_dir / 'results.png').exists():
    display(Image(filename=str(results_dir / 'results.png'), width=800))

## 3. Probar Modelo

In [ ]:
# Cargar mejor modelo
best_model = YOLO('runs/train/miniatures_obb/weights/best.pt')

# Probar en una imagen
test_images = list(Path(DATASET_PATH).rglob('*.jpg'))[:5]
if test_images:
    results = best_model(test_images[0], conf=0.5)
    
    # Mostrar resultado
    for r in results:
        im = r.plot()
        from PIL import Image as PILImage
        display(PILImage.fromarray(im[..., ::-1]))
        
        # Mostrar detecciones con ángulo
        if r.obb is not None:
            for i, (xyxyxyxy, conf, cls) in enumerate(zip(r.obb.xyxyxyxy, r.obb.conf, r.obb.cls)):
                angle = float(r.obb.xywhr[i][4]) * 180 / 3.14159  # Radianes a grados
                print(f"Figurita #{i+1}: confianza={conf:.2f}, ángulo={angle:.1f}°")

## 4. Exportar para Servidor

In [ ]:
# Copiar modelo al proyecto
import shutil

src = Path('runs/train/miniatures_obb/weights/best.pt')
dst = Path('../miniatures_obb.pt')

if src.exists():
    shutil.copy(src, dst)
    print(f"✅ Modelo copiado a: {dst.absolute()}")
    print(f"   Tamaño: {dst.stat().st_size / 1e6:.1f} MB")
else:
    print("❌ No se encontró el modelo entrenado")

In [ ]:
# Opcional: Exportar a ONNX para mejor rendimiento en CPU
best_model.export(
    format='onnx',
    imgsz=320,        # Más pequeño = más rápido
    simplify=True,    # Optimizar grafo
    dynamic=False,    # Tamaño fijo para mejor rendimiento
)

# Copiar ONNX
onnx_src = Path('runs/train/miniatures_obb/weights/best.onnx')
onnx_dst = Path('../miniatures_obb.onnx')
if onnx_src.exists():
    shutil.copy(onnx_src, onnx_dst)
    print(f"✅ ONNX copiado a: {onnx_dst.absolute()}")
    print(f"   Tamaño: {onnx_dst.stat().st_size / 1e6:.1f} MB")

## 5. Siguiente paso

Ahora copia `miniatures_obb.pt` a tu servidor y modifica `frame_processor.py` para usarlo:

```python
# En frame_processor.py, cambiar:
self.model = YOLO("miniatures_obb.pt")  # Tu modelo personalizado
```